## Formula 1 Hardest Tracks 2014-2024 Data Analysis

# Table of Contents
 - [Analysis Summary](#analysis-summary)
 - [Data Overview](#data-overview)
 - [Data Preperation](#mechanical-or-physical-dnfs-by-track)
 -  [Data Transformation](#Data-Transformation)
 - [Analysis](#Analysis)
 - [Conclusion](#Conclusion)
    - [Hardest Tracks On The Cars](#Hardest-Tracks-On-The-Cars)
    - [Hardest Tracks For The Drivers](#Hardest-Tracks-For-The-Drivers)
    - [Yearly Patterns](#Yearly-Patterns)

### Analysis Summary
The focus of this analysis is to gain insights into which tracks are the hardest on both the driver and car in Formula One (F1). The variables used to conduct this analysis are based on the cause of DNF (did not finish) from every race spanning from 2014 to 2024. We will also discuss any elements the identified tracks share to determine if there are set factors that make a race track difficult.
Data was downloaded from Kaggle that in turn was sourced directly from Formula One. Using SQl the data was cleaned and transformed before being transferred into tableau to create visuals. 
The main patterns identified were:
- Tracks featuring early in the season or located in harsher climates (e.g., hotter temperatures or higher altitudes) were harder on the cars. These tracks produced more mechanical or physical fault DNFs than any other
- Street circuits with limited space and tracks that promote more wheel-to-wheel racing were harder for the drivers. These races led to higher rates of collisions and DNFs due to driver error.
- Reliability issues were more common at the beginning of new regulatory eras, where teams were adapting to significant technical changes.


### Data Overview
The dataset used in this analysis was downloaded from Kaggle (https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020). This dataset was sourced directly from official Formula One records. Four data tables were downloaded and joined to conduct the analysis:
- Results – Each individual race entry, including finishing position, coded result status, and related race information
- Circuits – Each unique circuit with its corresponding ID and location
- Races – Every race with a unique ID, associated Circuit ID, and general race details
- tatus Key – A translation table converting coded result statuses into readable text

As Formula One has been active since 1950, I chose to focus on the most recent decade (2014–2024). There were two main reasons for this: first, reliability has become a much more critical focus for teams in recent years; and second, this period aligns with when I began following the sport as a fan, making the context more personally relevant and easier to interpret.


### Data Preparation 
All the data was checked and transformed using BigQuery. As with any data analysis project, the first step was to inspect, clean, and validate the dataset.
I noticed early on that some fields expected to contain integers instead had string values. This issue became apparent when attempting to upload the dataset to BigQuery using the autodetect schema option, which flagged schema mismatches. These string entries were primarily due to coded status values that required translation using the status key table. Rather than forcing these into integer format prematurely, I kept the values as strings, to be formatted properly, and then used this to create a new, cleaner version of the table.
I used the following code to check a specific entry I knew contained multiple string values were there should have been integers.


```sql
SELECT
  DISTINCT position, position_text
FROM `centered-binder-449311-f8.F1.Results`
ORDER BY position
```



The next thing I wanted to check is if all values that were supposed to be integers were formatted correctly. To do this I SAFE_CAST all integer columns as integers, then asked SQL to return any null values. This was to see if there were any entries that needed to be reformatted. The code I used for this was:

```sql
SELECT *
FROM (
SELECT 
SAFE_CAST(grid AS INT64) AS grid_position,
SAFE_CAST(positionOrder AS INT64) AS finish_position,
SAFE_CAST(resultId AS INT64) AS result_id,
SAFE_CAST(raceId AS INT64) AS race_id,
SAFE_CAST(driverId AS INT64) AS driver_id,
SAFE_CAST(constructorId AS INT64) AS constructor_id,
SAFE_CAST(number AS INT64) AS race_number,
grid, positionOrder, resultId, raceId, driverId, constructorId, number  
FROM 
`centered-binder-449311-f8.F1.Results`
)
WHERE grid_position IS NULL
OR finish_position IS NULL
OR result_id IS NULL
OR race_id IS NULL
OR driver_id IS NULL
OR constructor_id IS NULL
OR race_number IS NULL
```


No null values were returned. This validation step was repeated for each table in the dataset.

The final step in checking the data was to identify any duplicate entries. The most reliable way to do this was by checking the uniqueness of the resultId column. To check for duplicates, I used the following code:


```sql
SELECT resultId, COUNT(*) as count
FROM `centered-binder-449311-f8.F1.Results`
GROUP BY resultId
HAVING COUNT(*) > 1
```

This process was then repeated for each of the remaining three tables. All results were documented in a logbook to maintain a clear record of progress and any changes made during cleaning.

### Data Transformation

The first step in transforming the data was to join the Results table with the Races table. This was done to bring in the year of each race, allowing me to create a new table filtered to only include the last 10 years. The reason this step came first was to reduce the dataset size early in the process, so that when it came time to translate the status key, there were fewer entries to process. To achieve this, I ran the following query:


```sql
SELECT 
r.*, 
ra.year, 
ra.name AS race_name, 
ra.circuitId
FROM `centered-binder-449311-f8.F1.Results` r
JOIN `centered-binder-449311-f8.F1.Races` ra
ON r.raceId = ra.raceId
```

This was saved as a new table then filtered down to only show results from 2014-2024 using the following code. 

```sql
SELECT *
FROM `centered-binder-449311-f8.F1.Races_with_year`
WHERE 
year > 2013 
```

This reduced the number of entries from over 26,000 to just 4,600 and cut the number of distinct status key entries from 141 to 66.

Next, I needed to translate the statusId column into readable result descriptions. The statusId values were numerically coded, with their definitions stored in a separate lookup table. To begin this process, I identified which status codes appeared in races from the last ten years. To do this, I ran the following query:


```sql
SELECT DISTINCT
`status id`
FROM `centered-binder-449311-f8.F1.races_with_year`
ORDER BY `status id`
```

I recorded each distinct status ID along with its corresponding result definition—for example, a status ID of 1 indicated that the driver finished the race, while 2 indicated disqualification. I then grouped results with similar characteristics using the following query:

```sql
SELECT 
*,
CASE
WHEN `status id` = 1 THEN 'Finish'
WHEN `status id` = 2 THEN 'Disqualification'
WHEN `status id` IN (3, 20) THEN 'Accident'
WHEN `status id` IN (4, 130) THEN 'Collision' 
WHEN `status id` BETWEEN 21 AND 79 THEN 'Mechanical or Physical Fault'
WHEN `status id` IN (5, 6, 7, 8, 9, 10, 21, 79, 84, 95, 101, 103,105, 131, 132, 135, 136,141) THEN 'Mechanical or Physical Fault' 
WHEN `status id` BETWEEN 11 AND 19 THEN 'Behind on Laps'
WHEN `status id` = 137 THEN 'Damage Related'
WHEN `status id` = 138 THEN 'Damage Related'
WHEN `status id` IN (139, 96) THEN 'Other'
WHEN `status id` = 140 THEN 'Damage Related'
ELSE 'Unknown' 
END AS race_results
FROM `centered-binder-449311-f8.F1.race_reults_last_10years`;
```

Now that I had a clean table, filtered to the desired time frame and with race results converted to readable strings, I was able to import the dataset into Tableau to begin creating visuals.

### Analysis
To begin, I looked at the total count of each race result across all tracks to get a broad view of the data. 

![](https://raw.githubusercontent.com/Robbie-Moore/F1_Graphs/main/Race%20Results%20per%20Track.png)

I added a count to the graph to make it easier to gauge the percentage that each race result contributes to each track’s total. As a proportion of all race outcomes, the three tracks most frequently associated with mechanical or physical faults were Australia, Austria, and Bahrain.

Bahrain has long been notorious for placing strain on cars due to extreme heat. Australia, a street circuit, presents its own challenges for both drivers and cars—narrow sections, close barriers, and unpredictable weather conditions during that time of year all contribute to its difficulty. Australia and Bahrain often feature as the first or early season races, when teams are still fine-tuning their cars and gathering performance data, making mechanical failures more likely. The Austrian Grand Prix circuit is a fast track and sits at the highest altitude of all the European races. This adds complexity when balancing cooling and aerodynamic performance, and if not optimised, can lead to mechanical issues. 

The three tracks with the highest number of collisions are the British Grand Prix, with 0.39% of all race starts ending in a DNF due to collision, followed by Monaco (0.28%) and Brazil (0.26%). Both the British and Brazilian circuits promote intense wheel-to-wheel racing and are also known for unpredictable weather, creating ideal conditions for collisions.

Other tracks that encourage close racing, such as China, Russia, Spain, and the United States Grand Prix in Texas, also recorded over 0.2% of DNFs caused by collisions.
Interestingly, Monaco ranks third despite being known for limited overtaking opportunities, suggesting that both tightly contested tracks and narrow street circuits contribute to higher collision rates.

When it comes to driver-induced accidents, Monaco stands out as the only track where over 0.3% of results ended in a DNF due to an accident. This is followed by Singapore and Belgium, both at 0.17%. All three are considered particularly demanding circuits.
Monaco is known for its tight layout and minimal margin for error. Singapore shares many of these characteristics but adds the extreme heat and humidity, which can lead to driver fatigue and errors. Finally, Belgium presents it's own challenges due to its fast, demanding sections and complex layoutBelgium presents a different challenge, with fast, flowing sections and a technically demanding layout.


I then narrowed this down and looked at the percentages of the DNFs only, excluding any results that including finishing.

![](https://raw.githubusercontent.com/Robbie-Moore/F1_Graphs/main/DNFs%20per%20Track.png)

This largely follows the same pattern. The Austrian, Australian, and Bahrain Grands Prix have the highest percentage of DNFs caused by mechanical or physical faults. The British, Brazilian, and Monaco circuits show the highest proportion of collisions.

This time, the American Grand Prix matches Singapore in terms of accident-related DNFs. Finally, Monaco, Singapore, and Belgium record the highest percentages of DNFs caused by driver accidents.

Next, I analysed the average fastest lap speed to see whether faster tracks correlated with a higher number of collisions, accidents, or mechanical faults. 

![](https://raw.githubusercontent.com/Robbie-Moore/F1_Graphs/main/Average%20Fastest%20Lap%20Speed.png)

The red line represents the average fastest lap speed across all races. The Italian Grand Prix (Monza) has the highest average lap speed. When compared with previous graphs, Monza does show a relatively high percentage of mechanical or physical faults, but it does not exhibit a high rate of collisions or accidents.
The next two fastest tracks, Saudi Arabia and Las Vegas, also do not appear in the top three for any DNF category. While the Austrian, Australian, Bahrain, and British Grands Prix all record above average fastest lap speeds, there does not appear to be a clear correlation between speed and the frequency of DNFs. 


Next, I analysed the average number of places gained or lost at each track to identify which circuits are the most challenging for drivers to maintain their position. To achieve this, I created a calculated field in Tableau that subtracted the finishing position from the starting position, then calculated the average for each track. 

![](https://raw.githubusercontent.com/Robbie-Moore/F1_Graphs/main/Average%20Places%20Gained%20or%20Lost.png)

The Dutch Grand Prix is the most notable outlier, with the highest average number of positions lost across all tracks. This is followed by the Turkish Grand Prix, which only appeared twice during the analysed time period—making its sample size smaller and less representative compared to other circuits. Miami ranks third in terms of average positions lost.

The final part of the analysis involved comparing DNF causes across different years, with a primary focus on mechanical and physical faults to evaluate car reliability over time. 

![](https://raw.githubusercontent.com/Robbie-Moore/F1_Graphs/main/Race%20Results%20per%20Year.png)

Regarding collisions, 2018 and 2021 stand out, with both years recording 0.5% of all race results ending in a collision. The data does not suggest a clear reason for this. In 2021, car reliability was generally higher, meaning more cars remained in the race for longer periods—increasing the likelihood of collisions. However, 2018 does not follow this trend. This is likely due to the highly competitive nature of the season and how closely matched the teams were in terms of performance.

During the first four years under the 2014 regulation changes (2014–2017), mechanical or physical faults accounted for approximately 2.5%–3% of total race results each year. The 2014 season marked a major shift, with hybrid technology introduced to Formula One for the first time. In 2018, the number of mechanical or physical faults declined but remained relatively high. From there, it continued to decrease until 2021. In 2022, the frequency of these faults spiked again due to another significant regulatory overhaul. The aerodynamic rules were revised, and the cars became larger, heavier, and operated very differently from their predecessors. This presented new challenges for many teams. After the first season under these new regulations, the number of mechanical or physical faults began to decline once more. The trend line for these DNFs in the following graph strongly supports this pattern.

![](https://raw.githubusercontent.com/Robbie-Moore/F1_Graphs/main/DNFs%20Per%20Year.png)


### Conclusions

#### Hardest Tracks On The Cars
Austria, Australia, and Bahrain stood out as the most demanding tracks for the cars. The main contributing factors appear to be climate conditions (such as altitude and temperature), as well as their position in the season calendar—with earlier races showing a higher frequency of mechanical faults.

#### Hardest Tracks For The Drivers
The infamous Monaco street circuit takes pole position as the toughest track for drivers, despite being the slowest on the calendar. It is the only circuit to appear in the top three for both collisions and driver error–induced accidents. Other challenging tracks include the British, Brazilian, Singapore, and Belgian Grands Prix. This group features three fast, “racey” circuits and two tight, technical street tracks. The Dutch Grand Prix also deserves special mention for having the highest average positions lost, making it particularly difficult for drivers to hold position.

#### Yearly Patterns
Across DNFs caused by collisions or driver errors, no clear year-on-year trend emerges. Since the calendar remains relatively consistent, the only significant variable is the level of competitiveness between drivers and teams in each season. However, a clear pattern does emerge in relation to regulation changes: at the start of a new regulatory era, there is typically a spike in DNFs caused by mechanical or physical faults.


SANDBOZ

In [ ]:
I extracted all SQL snippets from the notebook’s markdown code blocks (no SQL magics or read_sql calls were found in code cells).

----- SQL #1 -----
SELECT
DISTINCT position, position_text
FROM centered-binder-449311-f8.F1.Results
ORDER BY position

----- SQL #2 -----
SELECT *
FROM (
SELECT
SAFE_CAST(grid AS INT64) AS grid_position,
SAFE_CAST(positionOrder AS INT64) AS finish_position,
SAFE_CAST(resultId AS INT64) AS result_id,
SAFE_CAST(raceId AS INT64) AS race_id,
SAFE_CAST(driverId AS INT64) AS driver_id,
SAFE_CAST(constructorId AS INT64) AS constructor_id,
SAFE_CAST(number AS INT64) AS race_number,
grid, positionOrder, resultId, raceId, driverId, constructorId, number
FROM
centered-binder-449311-f8.F1.Results
)
WHERE grid_position IS NULL
OR finish_position IS NULL
OR result_id IS NULL
OR race_id IS NULL
OR driver_id IS NULL
OR constructor_id IS NULL
OR race_number IS NULL

----- SQL #3 -----
SELECT resultId, COUNT() as count
FROM centered-binder-449311-f8.F1.Results
GROUP BY resultId
HAVING COUNT() > 1

----- SQL #4 -----
SELECT
r.*,
ra.year,
ra.name AS race_name,
ra.circuitId
FROM centered-binder-449311-f8.F1.Results r
JOIN centered-binder-449311-f8.F1.Races ra
ON r.raceId = ra.raceId

----- SQL #5 -----
SELECT *
FROM centered-binder-449311-f8.F1.Races_with_year
WHERE
year > 2013

----- SQL #6 -----
SELECT DISTINCT
status id
FROM centered-binder-449311-f8.F1.races_with_year
ORDER BY status id

----- SQL #7 -----
SELECT
*,
CASE
WHEN status id = 1 THEN 'Finish'
WHEN status id = 2 THEN 'Disqualification'
WHEN status id IN (3, 20) THEN 'Accident'
WHEN status id IN (4, 130) THEN 'Collision'
WHEN status id BETWEEN 21 AND 79 THEN 'Mechanical or Physical Fault'
WHEN status id IN (5, 6, 7, 8, 9, 10, 21, 79, 84, 95, 101, 103,105, 131, 132, 135, 136,141) THEN 'Mechanical or Physical Fault'
WHEN status id BETWEEN 11 AND 19 THEN 'Behind on Laps'
WHEN status id = 137 THEN 'Damage Related'
WHEN status id = 138 THEN 'Damage Related'
WHEN status id IN (139, 96) THEN 'Other'
WHEN status id = 140 THEN 'Damage Related'
ELSE 'Unknown'
END AS race_results
FROM centered-binder-449311-f8.F1.race_reults_last_10years;

Want me to save these into a .sql file or help convert them to pandas/CSV-based code using the Kaggle F1 CSVs?